# W261 Final Project
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 201`__ 


Alla Hale, Armand Kok, Daniel Olmstead, Adam Yang

The analysis below is a Click Through Rate prediction on the Criteo advertising data made public as part of a [Kaggle competition](https://www.kaggle.com/c/criteo-display-ad-challenge) in 2014.


# Notebook Set-Up

In [ ]:
# Imports 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time

from pyspark.sql import SQLContext
from pyspark.sql import types
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, StandardScaler, VectorIndexer, Normalizer
from pyspark.ml.linalg import VectorUDT
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, BinaryLogisticRegressionSummary
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [ ]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "hw5_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

# 2. Algorithm Explanation

## Logistic Regression

In this work, we will be implementing a logistic regression model to predict the click-through-rate based on the data provided by Criteo Labs.

The probability, $p$, of belonging to a given class is given by equation 1, where $\mathbf{w}$ is the vector of weights, $\mathbf{x}$ is the vector of observations, and $b$ is the bias term.

\begin{equation}\tag{1}
p=\frac{1}{1+\exp \left(-\mathbf{w}^{T} \cdot \mathbf{x}+b\right)}
\end{equation}

To estimate the weights for a logistic regression, we use maximum likelihood estimation and maximize the log likelihood

\begin{equation}\tag{2}
I(W) =\ln \prod_{i} P_{i}\\
I(W) =\ln \prod_{i}\left(\frac{1}{1+\exp \left(-\mathbf{w}^{T} \cdot \mathbf{x}_{i}+b\right)}\right)^{\frac{1+y_{i}}{2}}\left(1-\frac{1}{1+\exp \left(-\mathbf{w}^{T} \cdot \mathbf{x}_{i}+b\right)}\right)^{\frac{1-y_{i}}{2}}, 
\end{equation}

which is equal to minimizing the log loss function, $I(W)$, in equation 3, where $y$ is a label.

\begin{equation}\tag{3}
I(W)=\sum_{i} \log \left(1+\exp \left(-y \mathbf{w}^{T} \mathbf{x}_{i}\right)\right)
\end{equation}

Since this is a convex function, we can use gradient descent to find the vector, $\mathbf{w}$. The gradient is shown in vector notation in equation 4.

\begin{equation}\tag{4}
\nabla \mathbf{w}=-\sum_{i} y\left(1-\frac{1}{1+\exp \left(-y\left(\mathbf{w}^{T} \mathbf{x}_{i}+b\right)\right)}\right) \cdot \mathbf{x}_{i}
\end{equation}

The vector of weights is initially set to $\mathbf{0}$, and iteratively updated until convergence, according to equation 5.
\begin{equation}\tag{5}
\mathbf{w}=\mathbf{w}-\eta \cdot \nabla \mathbf{w}
\end{equation}

Equations for the $i^{th}$ component of the vector and the gradient, corresponding to equations 3, 4, and 5, are shown below in equations 6, 7, and 8, respectively.

\begin{equation}\tag{6}
I(W)=\sum_{i} Y^{l}\left(w_{0}+\sum_{i}^{n} w_{i} X_{i}^{l}\right)-\ln \left(1+\exp \left(w_{0}+\sum_{i}^{n} w_{i} X_{i}^{l}\right)\right)
\end{equation}

\begin{equation}\tag{7}
\frac{\partial l(W)}{\partial w_{i}}=\sum_{l} X_{i}^{l}\left(Y^{l}-\hat{P}\left(Y^{l}=1 | X^{l}, W\right)\right)
\end{equation}

\begin{equation}\tag{8}
w_{i} \leftarrow w_{i}+\eta \sum_{l} X_{i}^{l}\left(Y^{l}-\hat{P}\left(Y^{l}=1 | X^{l}, W\right)\right)
\end{equation}

Likely, we will regularize the gradient descent so that we do not overfit to our data. In order to do this, we introduce a regularization coefficient $\lambda$ multiplied either by the L1 norm (Lasso) or the L2 norm (Ridge).

The objective function and gradient are shown for lasso regression below.

Objective function

\begin{equation}\tag{9}
I(W)= \sum_{i} \log \left(1+\exp \left(-y\left(\mathbf{w}^{T} \mathbf{x}_{i}+b\right)\right)\right)+\lambda|\mathbf{w}|
\end{equation}

Gradient

\begin{equation}\tag{10}
\nabla \mathbf{w}=-\sum_{i}y\left(1-\frac{1}{1+\exp \left(-y\left(\mathbf{w}^{T} \mathbf{x}_{i}+b\right)\right)}\right) \cdot \mathbf{x}_{i}+\lambda \text{sign}(\mathbf{w})
\end{equation}

And again for ridge regression below.

Objective function

\begin{equation}\tag{11}
I(W)= \sum_{i} \log \left(1+\exp \left(-y\left(\mathbf{w}^{T} \mathbf{x}_{i}+b\right)\right)\right)+\lambda \mathbf{w}^{2}
\end{equation}

Gradient

\begin{equation}\tag{12}
\nabla \mathbf{w}=-\sum_{i}y\left(1-\frac{1}{1+\exp \left(-y\left(\mathbf{w}^{T} \mathbf{x}_{i}+b\right)\right)}\right) \cdot \mathbf{x}_{i}+\lambda \mathbf{w}
\end{equation}

In all cases, the update to $\mathbf{w}$ remains the same as shown in equation 5.

# 3. EDA and Discussion of Challenges

In [ ]:
# load the data
# Define the schema prior to loading the data
schema = StructType([StructField("label", IntegerType(), True),
                     StructField("I1", IntegerType(), True),
                     StructField("I2", IntegerType(), True),
                     StructField("I3", IntegerType(), True),
                     StructField("I4", IntegerType(), True),
                     StructField("I5", IntegerType(), True),
                     StructField("I6", IntegerType(), True),
                     StructField("I7", IntegerType(), True),
                     StructField("I8", IntegerType(), True),
                     StructField("I9", IntegerType(), True),
                     StructField("I10", IntegerType(), True),
                     StructField("I11", IntegerType(), True),
                     StructField("I12", IntegerType(), True),
                     StructField("I13", IntegerType(), True),
                     StructField("C1", StringType(), True),
                     StructField("C2", StringType(), True),
                     StructField("C3", StringType(), True),
                     StructField("C4", StringType(), True),
                     StructField("C5", StringType(), True),
                     StructField("C6", StringType(), True),
                     StructField("C7", StringType(), True),
                     StructField("C8", StringType(), True),
                     StructField("C9", StringType(), True),
                     StructField("C10", StringType(), True),
                     StructField("C11", StringType(), True),
                     StructField("C12", StringType(), True),
                     StructField("C13", StringType(), True),
                     StructField("C14", StringType(), True),
                     StructField("C15", StringType(), True),
                     StructField("C16", StringType(), True),
                     StructField("C17", StringType(), True),
                     StructField("C18", StringType(), True),
                     StructField("C19", StringType(), True),
                     StructField("C20", StringType(), True),
                     StructField("C21", StringType(), True),
                     StructField("C22", StringType(), True),
                     StructField("C23", StringType(), True),
                     StructField("C24", StringType(), True),
                     StructField("C25", StringType(), True),
                     StructField("C26", StringType(), True)])


In [ ]:
# Load toy data into dataframe
toy_df = spark.read.parquet("toyData/*.parquet")

In [ ]:
# Show toy_data
pd.DataFrame(toy_df.take(30), columns=toy_df.columns).transpose()

## Transform the data

# 4. Gradient Descent Approach to Logistic Regression

In [ ]:
# Just some code to get only the numeric numbers for training. Won't be using this for the official notebook
toy_rdd = toy_df.rdd.map(tuple)
toy_RDD = toy_rdd.map(lambda line: (line[0],(line[1:14]))).cache()

In [ ]:
dataset = spark.createDataFrame(
    [(1.0, 1, 1, 1),
     (1.0, 5, 2, 2),
     (0.0, 3, 0, 3)],
    ["label", "page_num", "hour", "id"])

In [ ]:
toy_rdd = dataset.rdd.map(tuple)
toy_RDD = toy_rdd.map(lambda line: (line[0],(line[1:4]))).cache()

## 4.1 Log Loss

As shown in *equation 3*, the Log Loss equation, $I(W)$, is given as:

$$
I(W)=\sum_{i} \log \left(1+\exp \left(-y_i \mathbf{\theta}^{T} \mathbf{\bar{x}}_{i}\right)\right)
$$

where $\theta := \begin{bmatrix} b \\ w \end{bmatrix}$ and  $\bar{x}_i := \begin{bmatrix} 1 \\ x_i \end{bmatrix}$. Furthermore, $x_i$ represents each row of our feature inputs, $y_i$ represents the labels to each of the rows, and $w$ represents the coefficient vector of our model. The $\theta$ and $\bar{x}_i$ notations allow us to incorporate the intercept of the logistic regression model into the loss function. If we leave out the intercept of the model, we will be claiming that when all the predictors are equal to 0, the resulting $logit(p)$ will be 0. Since $logit(p) = log(\frac{p}{1-p})$, if we set $logit(p) = 0$, we subsequently get: 
$$log(\frac{p}{1-p}) = 0$$
$$e^0 = \frac{p}{1-p} = 1$$
$$\therefore p = 0.5$$
This means that if we leave out the intercept of the model, we will be claiming that when all the predictors are zero, the probability of getting a click on the ad will be 0.5. This is not a claim that we can make so it is very important that we include the intercept into our logistic regression.
Below is a function that calculates the Log Loss of a given model.

In [ ]:
def LogLoss(dataRDD, W):
    """
    Compute the Log Loss of our model.
    Args:
        dataRDD - each record is a tuple of (y, features_array)
        W       - (array) model coefficients
    """
    # Add 1 to the front of the predictors array
    # Note that the b value has to be added to the front of our theta array
    augmentedData = dataRDD.map(lambda x: (x[0], np.append([1.0], x[1])))

    def LogLossPerRow(line):
        # Calculate log(1 + exp(-ywx)) for each row of the data in parallel
        actual_y, features = line
        predicted_y = np.dot(np.transpose(W),features)
        yield np.log(1.0 + np.exp(-1.0*actual_y*predicted_y))
    
    loss = augmentedData.flatMap(LogLossPerRow).sum()

    return loss

Now lets test our LogLoss function by setting b to the mean of the our label values and all our predictors to 0.

In [ ]:
# Calculate the mean of all our label values
meanQuality = toy_RDD.map(lambda x: x[0]).mean()
print(f"Mean: {meanQuality}")

In [ ]:
# Set the baseline model so that b is the mean we calculated and all other features are 0
#BASELINE = np.append([meanQuality],[0,0,0,0,0,0,0,0,0,0,0,0,0])
BASELINE = np.append([meanQuality],[0,0,0])

In [ ]:
# Compute the loss of the baseline model
assert len(BASELINE) == len(toy_RDD.take(1)[0][1]) + 1, "Double check model dimensions"
print(f"Baseline model loss: {LogLoss(toy_RDD, BASELINE)}")

In [ ]:
# Verify if our result is correct or not

# Count how many 1's and 0's we have in our label
ones = toy_RDD.map(lambda line: 1 if line[0] == 1 else 0).sum()
zeros = toy_RDD.count() - ones
print("There are {} ones and {} zeros in the label column of our dataset".format(ones, zeros))

# Calculate the LogLoss in our example case
loss = ones*np.log(1.0 + np.exp(-1.0*meanQuality)) + zeros*np.log(1.0 + np.exp(0))
print("The loss of the baseline model through manual calculation is {}".format(loss))

We see that the results of our LogLoss() function is correct.

## 4.2 Gradient Descent

Now that we have a way to find the Log Loss of our model, we will approach finding the logistic regression model through gradient descent. Equations 4, 10, and 12, gives us the gradients for a vanilla gradient descent, L1 regularization (lasso), and L2 regularization (ridge) respectively. These gradient equations are provided below in the same order.
$$
\nabla \mathbf{w}=-\sum_{i} y_i\left(1-\frac{1}{1+\exp \left(-y_i \mathbf{\theta}^{T} \mathbf{\bar{x}}_{i}\right)}\right) \cdot \mathbf{x}_{i}
$$

$$
\nabla \mathbf{w}=-\sum_{i}y_i\left(1-\frac{1}{1+\exp \left(-y_i \mathbf{\theta}^{T} \mathbf{\bar{x}}_{i}\right)}\right) \cdot \mathbf{x}_{i}+\lambda \text{sign}(\mathbf{w})
$$


$$
\nabla \mathbf{w}=-\sum_{i}y_i\left(1-\frac{1}{1+\exp \left(-y_i \mathbf{\theta}^{T} \mathbf{\bar{x}}_{i}\right)}\right) \cdot \mathbf{x}_{i}+\lambda \mathbf{w}
$$

With these gradients, we can iteratively update our model with the following equation until it converges:

$$
\mathbf{w_{new}}=\mathbf{w_{old}}-\eta \cdot \nabla \mathbf{w}
$$

The code below will be caculating the gradient, whether it is vanilla, lasso, or ridge, depending on the user's choice. Then the gradient will be used to update the coefficients of our model.

In [ ]:
# This code calculates the 3 different gradients with and without regularization
# Then it updates the model (w) and outputs the new model
def GDUpdate_wReg(dataRDD, W, learningRate = 0.1, regType = None, regParam = 0.1):
    """
    Perform one gradient descent step/update with ridge or lasso regularization.
    Args:
        dataRDD - tuple of (y, features_array)
        W       - (array) model coefficients with intercept at index 0
        learningRate - (float) defaults to 0.1
        regType - (str) 'ridge' or 'lasso', defaults to None
        regParam - (float) regularization term coefficient
    Returns:
        model   - (array) updated coefficients, intercept still at index 0
    """
    # augmented data
    augmentedData = dataRDD.map(lambda x: (x[0], np.append([1.0], x[1])))
    
    new_model = None

    def GradientPerRow(line):
        # Calculates -y(1- 1/ 1+ exp( -ywx))x for each row
        true_y, features = line
        predicted_y = np.dot(np.transpose(W),features)
        yield -1.0*true_y*(1.0 - 1.0/(1.0 + np.exp(-1.0*true_y*predicted_y)))*features
        
    # Use the same way as before to find the first component of the gradient function
    grad = augmentedData.flatMap(GradientPerRow).sum()
    
    # Take out the bias stored in index 0 of W
    model = W[1:]
    
    # Figure out the regulation component
    if regType == None:
        pass
        
    elif regType == 'lasso':
        reg_comp = regParam*np.sign(model)
        # Update the gradient function by taking the regularization component into consideration
        grad = grad + np.append(0,reg_comp)
                
    elif regType == 'ridge':
        reg_comp = regParam*model
        # Update the gradient function by taking the regularization component into consideration
        grad = grad + np.append(0,reg_comp)
    
    new_model = W - (learningRate*grad)
    
    return new_model

In [ ]:
GDUpdate_wReg(toy_RDD, BASELINE, learningRate = 0.1, regType = None, regParam = 0.1)
#prob = 1.0/(1.0 + np.exp(-1.0*np.dot(updated_model, np.append(1,pd.DataFrame(toy_df_transformed.collect(), columns=toy_df_transformed.columns)['features'][2]))))
#pd.DataFrame(toy_df_transformed.collect(), columns=toy_df_transformed.columns)['features'][0]
#prob

In [ ]:
GDUpdate_wReg(toy_RDD, [0.73451539, 0.20354618, 1.01773089, 0.50886545], learningRate = 0.1, regType = None, regParam = 0.1)

In [ ]:
m = [0.73451539, 0.20354619, 1.01773091, 0.50886546]
np.exp(-np.dot(m, [1,1,18,7]))

In [ ]:
-(1- 1/(1+ np.exp(-2/3)))*15

In [ ]:
-(1- 1/(1+ np.exp(0)))

Now that we have a way to calculate the gradient and update our model coefficients, we can not perform gradient descent by iterating through this method until our Log Loss converges towards a minimum.

In [ ]:
# This code performs the Gradient Descent iterations 
def GradientDescent_wReg(trainRDD, wInit, nSteps = 20, learningRate = 0.1,
                         regType = None, regParam = 0.1, verbose = False):
    """
    Perform nSteps iterations of regularized gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each update
    model = wInit
    for idx in range(nSteps):  
        # update the model
        model = GDUpdate_wReg(trainRDD, model, learningRate, regType, regParam)
        
        # keep track of test/train loss for plotting
        train_history.append(LogLoss(trainRDD, model))
        #test_history.append(LogLoss(testRDD, model))
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training loss: {LogLoss(trainRDD, model)}")
            print(f"test loss: {LogLoss(testRDD, model)}")
            print(f"Model: {[round(w,3) for w in model]}")
    return train_history, test_history, model_history

In [ ]:
# run 50 iterations
wInit = BASELINE
#trainRDD, testRDD = toy_RDD.randomSplit([0.8,0.2], seed = 5)
start = time.time()
ridge_results = GradientDescent_wReg(toy_RDD, wInit, nSteps = 1000, learningRate = 0.1, 
                                     regType=None, regParam = 5.0, verbose = True )
print(f"\n... trained {len(ridge_results[2])} iterations in {time.time() - start} seconds")

In [ ]:
#toy_df_label = toy_df.select('label')
#toy_df_features = toy_df.select('I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13')


assembler = VectorAssembler(
    inputCols=['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13'],
    outputCol="features")

toy_df_transformed = assembler.transform(toy_df)
toy_df_transformed = toy_df_transformed.select('features','label')
pd.DataFrame(toy_df_transformed.take(30), columns=toy_df_transformed.columns).transpose()

In [ ]:
assembler = VectorAssembler(
    inputCols=["page_num", "hour", "id"],
    outputCol="features")
toy_df_transformed = assembler.transform(dataset)
toy_df_transformed = toy_df_transformed.select('features','label')
pd.DataFrame(toy_df_transformed.take(30), columns=toy_df_transformed.columns).transpose()

In [ ]:
lr = LogisticRegression(maxIter=50, regParam=0, standardization=False, fitIntercept=True)
lrModel = lr.fit(toy_df_transformed)

In [ ]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

In [ ]:
predictions_df = lrModel.transform(toy_df_transformed)
pd.DataFrame(predictions_df.take(30), columns=predictions_df.columns).transpose()

In [ ]:
# plot error curves - RUN THIS CELL AS IS
def plotErrorCurves(trainLoss, testLoss, title = None):
    """
    Helper function for plotting.
    Args: trainLoss (list of MSE) , testLoss (list of MSE)
    """
    fig, ax = plt.subplots(1,1,figsize = (16,8))
    x = list(range(len(trainLoss)))[1:]
    ax.plot(x, trainLoss[1:], 'k--', label='Training Loss')
    ax.plot(x, testLoss[1:], 'r--', label='Test Loss')
    ax.legend(loc='upper right', fontsize='x-large')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Mean Squared Error')
    if title:
        plt.title(title)
    plt.show()

In [ ]:
# part d - save and display ridge results (RUN THIS CELL AS IS)
trainLoss, testLoss, models = ridge_results
plotErrorCurves(trainLoss, testLoss, title = 'Ridge Regression Error Curves' )

# 4. Algorithm Implementation

$$
\nabla \mathbf{w}=\sum_{i} \left(\frac{1}{1+\exp \left(- \mathbf{\theta}^{T} \mathbf{\bar{x}}_{i}\right)}-y_i\right) \cdot \mathbf{x}_{i}
$$

# 5. Application of Course Concepts